In [1]:
import networkx as nx

import numpy as np
from basicanf import BasicANF


In [2]:
import csv

G = nx.random_regular_graph(3, 2000)
edges = list(G.edges())
edges.extend([(y, x) for x, y in G.edges()])

print(edges[0])

with open('edgelist.csv','w', newline='') as out:
    csv_out = csv.writer(out)
    csv_out.writerows(edges)

max_distance = 5


(965, 1209)


In [3]:
basicAnf = BasicANF(edges, 9, k=128)

basicAnf._test_generate_bitmask()

100%|██████████| 100000/100000 [00:03<00:00, 29833.81it/s]

0 0.49936
1 0.25025
2 0.12584
3 0.06171
4 0.03131
5 0.01535
6 0.00762
7 0.0042
8 0.00222
9 0.00096
10 0.00055
11 0.00033
12 0.00015
13 7e-05
14 4e-05
15 3e-05
16 0.0
17 0.0
18 0.0
19 1e-05


In [4]:
d, IN, M = basicAnf.compute(max_distance=max_distance)
d[0] = 1
d


{1: 14.235710111672988,
 2: 86.04080061301725,
 3: 419.95726306892243,
 4: 1242.9554948427055,
 0: 1}

In [10]:
M[2][1]

array([ True,  True,  True, ..., False, False, False])

In [11]:
def neighborhood(G, node, n):
    path_lengths = nx.single_source_dijkstra_path_length(G, node, n)
    return path_lengths

neighs = {node: neighborhood(G, node, 5) for node in G.nodes()}

nx_exact_neighbourhood = {h: np.mean([len([n for n, l in neighs[node].items() if l <= h]) for node in G.nodes()]) for h in range(max_distance)}

nx_exact_neighbourhood

{0: 1.0, 1: 4.0, 2: 9.982, 3: 21.898, 4: 45.503}

In [13]:
import snap

G1 = snap.TUNGraph.New()

for n in G.nodes():
  G1.AddNode(n)

for s, t in G.edges():
  G1.AddEdge(s, t)

DistNbrsV = snap.TIntFltKdV()

snap.GetAnf(G1, DistNbrsV, max_distance, False, 128)

for i in DistNbrsV:
  print(i.Dat())

snap_neighbourhoods = {h: item.Dat() / G.number_of_nodes() for h, item in zip(range(max_distance), DistNbrsV)}

snap_neighbourhoods

2000.0
9010.075839197994
21056.56432167854
44904.25400606593
92557.98630043303


{0: 1.0,
 1: 4.505037919598998,
 2: 10.52828216083927,
 3: 22.452127003032967,
 4: 46.278993150216515}

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

plt.title("Error in neighborhood approximation")
plt.ylabel("Error relative to exact")
plt.xlabel("Distance")
plt.tight_layout()
plt.xticks(range(max_distance))

snap_results = np.array(list(nx_exact_neighbourhood.values())) - np.array(list(snap_neighbourhoods.values()))
ANF_results = np.array([d[h] for h in range(max_distance)]) - np.array(list(snap_neighbourhoods.values()))

sns.lineplot(x=range(max_distance), y=snap_results, label='Snap')
sns.lineplot(x=range(max_distance), y=ANF_results, label='ANF')